In [2]:
from googleapiclient.discovery import build
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax
import torch


In [ ]:

# Your API key here 
api_key = 'YOUR API KEY'

# Build the service object
youtube = build('youtube', 'v3', developerKey=api_key)

# Replace 'VIDEO_ID' with the ID of the video you want comments from
video_id = 'pEskP0ulPlA'

# Request the comments
comments = youtube.commentThreads().list(
    part='snippet',
    videoId=video_id,
    maxResults=100,  # Limit to 100 comments
    textFormat='plainText'
).execute()

# List to hold the comments
comments_list = []

# Iterate through the comments and extract the required info
for item in comments['items']:
    comment = item['snippet']['topLevelComment']['snippet']
    comments_list.append([comment['authorDisplayName'], comment['publishedAt'], comment['textDisplay']])

# Create a DataFrame
comments_df = pd.DataFrame(comments_list, columns=['User', 'Date', 'Comment'])


In [ ]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

In [ ]:
# Function to get sentiment
def get_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    probs = softmax(outputs.logits, dim=1)
    sentiment = torch.argmax(probs, dim=1).numpy()[0]  # 0: Negative, 1: Neutral, 2: Positive
    return ['-1', '0', '1'][sentiment]

# Apply sentiment analysis on the 'Comment' column
comments_df['Sentiment'] = comments_df['Comment'].apply(get_sentiment)


In [17]:
comments_df.to_csv('youtube_comments.csv', index=False)
